# Seasonal cycles of parameters within the mixed layer

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import os
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import stats
from tqdm import tqdm
import xarray as xr

In [1]:
# Set directories
base_dir = "/Users/smb-uh/"
data_dir = base_dir + "UHM_Ocean_BGC_Group Dropbox/Datasets/"
home_dir = base_dir + "UHM_Ocean_BGC_Group Dropbox/Seth Bushinsky/Work/"
figure_dir = home_dir + "Projects/2025_10_BGC_Argo_Plus_Code_examples/plots/"
argo_path = data_dir + "Data_Products/BGC_ARGO_GLOBAL/2025_01_24/processed/"
glodap_path = data_dir + "Data_Products/GLODAP/"
plot_ver = 'v_1'

# Float observations

In [4]:
argolist = []
for file in os.listdir(argo_path):
    if file.endswith('Sprof_BGCArgoPlus_full.nc'):
        argolist.append(file)

print(len(argolist))

2225


## Load float data, calculate ML averages, save parameters if they exist

In [5]:
# let's try separate xarrays for each variable? Since we'll have wildly different amounts of observations
var = 'NITRATE_ADJUSTED_BGCArgoPlus' # need to set both variable and which version (raw, adjusted, bgcargo+)
var_all = {}

for n in tqdm(range(0, len(argolist))):
    try:
        argo_n = xr.load_dataset(argo_path+argolist[n])
        # argo_n = argo_n.set_coords(('PRES_ADJUSTED','LATITUDE','LONGITUDE','JULD'))
    except:
        print(argolist[n] + ' failed to load')
        continue

    # check for presence of var:
    var_list = list(argo_n.data_vars)
    if var not in var_list: 
        continue
    
    # check if var is non-nan
    if np.sum(~np.isnan(argo_n[var]))==0:
        continue 
    # calculate ML averages
    ml_var = np.zeros(len(argo_n.N_PROF))
    ml_var[:] = np.nan

    for idx, p in enumerate(argo_n.N_PROF.values):
        var_prof = argo_n[var][p,argo_n['PRES_ADJUSTED'][p,:]<=argo_n.MLD[p]]
        if np.sum(~np.isnan(var_prof))==0:
            continue
        ml_var[p] = np.nanmean(var_prof)
        
    # print(ml_var)

    # identify non-nan ml averages, save those and metadata into a new xarray
    non_nan_index = ~np.isnan(ml_var)
    valid_len = len(ml_var[non_nan_index])    
    wmo_n = argo_n.PLATFORM_NUMBER.values.astype(int)[0]

    ml_var_n = xr.Dataset()
    ml_var_n['wmo']=(['N_PROF'],np.repeat(wmo_n,valid_len))
    ml_var_n['LATITUDE'] = (['N_PROF'], argo_n['LATITUDE'][non_nan_index].data)
    ml_var_n['LONGITUDE'] = (['N_PROF'], argo_n['LONGITUDE'][non_nan_index].data)
    ml_var_n['juld'] = (['N_PROF'],argo_n['JULD'][non_nan_index].data)
    ml_var_n[var]  = (['N_PROF'],ml_var[non_nan_index])

    # # append all float into one long xarray
    if var in var_all.keys():
        var_all[var] = xr.concat([var_all[var], ml_var_n], 'N_PROF') 
    else:
        var_all[var] = ml_var_n

    if n>20:
        break

  1%|          | 21/2225 [00:06<12:06,  3.03it/s]


# Load GLODAP data

In [8]:
# 
gdap = xr.open_dataset(glodap_path + 'GLODAPv2.2023_Merged_Master_File.nc')
gdap

<xarray.Dataset> Size: 2GB
Dimensions:           (index: 1402829)
Coordinates:
  * index             (index) int64 11MB 0 1 2 3 ... 1402826 1402827 1402828
Data variables: (12/117)
    G2expocode        (index) <U12 67MB ...
    G2cruise          (index) float64 11MB ...
    G2station         (index) float64 11MB ...
    G2region          (index) float64 11MB ...
    G2cast            (index) float64 11MB ...
    year              (index) float64 11MB ...
    ...                ...
    spice             (index) float64 11MB ...
    sigma0            (index) float64 11MB ...
    gamma             (index) float64 11MB ...
    obs_index         (index) int64 11MB ...
    DOXY_SAT          (index) float64 11MB ...
    MLD               (index) float64 11MB ...

In [7]:
gdap

<xarray.Dataset> Size: 2GB
Dimensions:           (index: 1402829)
Coordinates:
  * index             (index) int64 11MB 0 1 2 3 ... 1402826 1402827 1402828
Data variables: (12/117)
    G2expocode        (index) <U12 67MB ...
    G2cruise          (index) float64 11MB ...
    G2station         (index) float64 11MB ...
    G2region          (index) float64 11MB ...
    G2cast            (index) float64 11MB ...
    year              (index) float64 11MB ...
    ...                ...
    spice             (index) float64 11MB ...
    sigma0            (index) float64 11MB ...
    gamma             (index) float64 11MB ...
    obs_index         (index) int64 11MB ...
    DOXY_SAT          (index) float64 11MB ...
    MLD               (index) float64 11MB ...

# Maps of data 

In [ ]:
# First, let's see what observations we have globally:

# Separate data by region

In [ ]:
# Load masks - ideally this step should be possible with different sets of masks 
# for now use geographical boundaries, start w/ SO
